### Install tendb
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ tendb
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb/dist/tendb-0.0.1-py3-none-any.whl
```

Read Arrow table

In [1]:
import pyarrow as pa
from pyarrow import csv

In [2]:
csv_options = pa.csv.ParseOptions(delimiter='|')
lineitem_table = pa.csv.read_csv(input_file='lineitem.tbl', parse_options=csv_options)
lineitem_table

pyarrow.Table
L_ORDERKEY: int64
L_PARTKEY: int64
L_SUPPKEY: int64
L_LINENUMBER: int64
L_QUANTITY: int64
L_EXTENDEDPRICE: double
L_DISCOUNT: double
L_TAX: double
L_RETURNFLAG: string
L_LINESTATUS: string
L_SHIPDATE: timestamp[s]
L_COMMITDATE: timestamp[s]
L_RECEIPTDATE: timestamp[s]
L_SHIPINSTRUCT: string
L_SHIPMODE: string
L_COMMENT: string

In [3]:
len(lineitem_table)

6005

In [4]:
isinstance(lineitem_table, object)

True

In [5]:
hex(id(lineitem_table))

'0x7f689803e330'

TenDB sample commands

In [6]:
import tendb as ten

In [7]:
tcache = ten.new_cache()

In [8]:
tcache

671126160

In [ ]:
ten.add_table(tcache, 'lineitem', hex(id(lineitem_table)))

In [ ]:
ten.query6()

In [ ]:
ten.test()

In [ ]:
f = ten.Foo()

In [ ]:
f.bar()